In [1]:
import open3d as o3d
import numpy as np
import sys

sys.path.append('./data/')
from camera import *
from dataloader import *

sys.path.append('./algorithm/')
from measurement import *
from pose import *
from update import *
from prediction import *

from pipeline import *

import matplotlib.pyplot as plt

In [2]:
kf = KinectFusion()

KinectFusion pipeline total frame number: 798


In [3]:
kf.first_frame()

Running frame: 0


In [4]:
depth = kf.get_depth_map()
depth_numpy = np.asarray(depth)
depth_numpy = depth_numpy.reshape(depth_numpy.shape[:2])

In [5]:
kf.curr_point_cloud = point_cloud_from_depth(depth_numpy, kf.camera)

In [6]:
prev_points = np.asarray(kf.prev_point_cloud.points)

In [7]:
curr_points = np.asarray(kf.curr_point_cloud.points)

In [8]:
print(np.mean(prev_points))

0.31065216969356885


In [9]:
print(np.mean(curr_points))

0.30966519374999996


4984.114452756881